# Phishing Detection Using BERT- Fine-tuning notebook

## Table of Contents 
* Introduction
* List of datasets used
* Reading in the datasets
* Initialize CLX Phishing Detection and BERT model
* Training with three datasets CLAIR+SPAM_ASSASSIN+ENRON
* Evaluation of the Test Set of CLAIR+SPAM_ASSASSIN+ENRON Datasets
* References

## Introduction
Phishing is a method used by fraudsters/hackers to obtain sensitive information from email users by pretending to be from legitimate institutions/people.
Various machine learning methods are in use to detect and filter phishing/spam emails. 
In this notebook, we show how to train a *BERT language model and analyse the performance on various datasets. We have fine-tuned a pre-trained BERT model with a classification layer using HuggingFace library. 
*BERT stands for Bidirectional Encoder Representations from Transformers. The paper can be found [here.](https://arxiv.org/pdf/1810.04805.pdf)

## Datasets used
* [CLAIR-Fraudulent E-mail Corpus](https://www.kaggle.com/rtatman/fraudulent-email-corpus)
* [SPAM_ASSASSIN Dataset](https://spamassassin.apache.org/old/publiccorpus/)
* [Enron Emails](https://www.cs.cmu.edu/~enron/)

### Required Libraries

In [1]:
import cudf;
from cuml.model_selection import train_test_split;
from binary_sequence_classifier import BinarySequenceClassifier;
import argparse;
import s3fs;
from os import path;
import logging

## Reading the files

In [2]:
CLAIR_TSV = "Phishing_Dataset_Clair_Collection.tsv"
SPAM_TSV = "spam_assassin_spam_200_20021010.tsv"
EASY_HAM_TSV = "spam_assassin_easyham_200_20021010.tsv"
HARD_HAM_TSV = "spam_assassin_hardham_200_20021010.tsv"
ENRON_TSV = "enron_10000.tsv"
S3_BASE_PATH = "rapidsai-data/cyber/clx"


In [3]:
# Clair dataset
if not path.exists(CLAIR_TSV):
    fs = s3fs.S3FileSystem(anon=True)
    fs.get(S3_BASE_PATH + "/" + CLAIR_TSV, CLAIR_TSV)
    
dfclair = cudf.read_csv(CLAIR_TSV, delimiter='\t', header=None, names=['label', 'email']).dropna()

In [4]:
# Phishing emails of the SPAM ASSASSIN dataset
if not path.exists(SPAM_TSV):
    fs = s3fs.S3FileSystem(anon=True)
    fs.get(S3_BASE_PATH + "/" + SPAM_TSV, SPAM_TSV)
 
dfspam = cudf.read_csv(SPAM_TSV, delimiter='\t', header=None, names=['label', 'email'])

In [5]:
# Benign emails of the SPAM ASSASSIN dataset
if not path.exists(EASY_HAM_TSV):
    fs = s3fs.S3FileSystem(anon=True)
    fs.get(S3_BASE_PATH + "/" + EASY_HAM_TSV, EASY_HAM_TSV)
    
dfeasyham = cudf.read_csv(EASY_HAM_TSV, delimiter='\t', header=None, names=['label', 'email'])

In [6]:
# Benign emails of the SPAM ASSASSIN dataset that are easy to be confused with phishing emails
if not path.exists(HARD_HAM_TSV):
    fs = s3fs.S3FileSystem(anon=True)
    fs.get(S3_BASE_PATH + "/" + HARD_HAM_TSV, HARD_HAM_TSV)

dfhardham = cudf.read_csv(HARD_HAM_TSV, delimiter='\t', header=None, names=['label', 'email'])

In [7]:
# Benign Enron emails
if not path.exists(ENRON_TSV):
    fs = s3fs.S3FileSystem(anon=True)
    fs.get(S3_BASE_PATH + "/" + ENRON_TSV, ENRON_TSV)

dfenron = cudf.read_csv(ENRON_TSV, delimiter='\t', header=None, names=['label', 'email'])

In [8]:
# The files contain the first 200 words of each email. 

## Initialize/Load BERT model

We will initialize the sequence classifier module with a pre-trained BERT model. The pre-trained model we use is located at https://huggingface.co/bert-base-uncased For more information on the model, please see the paper at https://arxiv.org/pdf/2005.01634.pdf


In [9]:
seq_classifier = BinarySequenceClassifier()
seq_classifier.init_model("bert-base-uncased")

# init_model can also load pre-trained model by passing it model directory path

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

## Training with CLAIR+SPAM_ASSASSIN+ENRON datasets

Clair fraudulent e-mails contain criminally deceptive information, usually with the intent of convincing the recipient to give the sender a large amount of money. The original dataset can be found [here](https://www.kaggle.com/rtatman/fraudulent-email-corpus)

SPAM_ASSASIN public mail corpus consists of three classes. The first category is \"easy ham\" which has benign emails; the second category is \"hard ham\" which has emails which are benign but are closer to spam, and finally, the spam category that includes the fraudulent emails. More info [here](https://spamassassin.apache.org/old/publiccorpus/readme.html)

Enron Email Dataset was collected and prepared by the CALO Project (A Cognitive Assistant that Learns and Organizes). We use them as benign email class. 
More info on the ENRON Email Dataset can be found [here](https://www.cs.cmu.edu/~./enron/)

Merge all the datasets, split into training and test set and then tokenize the emails

In [10]:
df_total = cudf.concat([dfhardham,dfeasyham,dfspam,dfclair,dfenron],ignore_index=True)

In [11]:
X_train, X_test, y_train, y_test = train_test_split(df_total, 'label', train_size=0.8)

In [12]:
seq_classifier.train_model(X_train["email"], y_train, epochs=3)

Epoch:  33%|███▎      | 1/3 [02:34<05:08, 154.42s/it]

Train loss: 0.061520517534322125


Epoch:  67%|██████▋   | 2/3 [05:08<02:33, 154.00s/it]

Train loss: 0.011529590881415499


Epoch: 100%|██████████| 3/3 [07:42<00:00, 154.12s/it]

Train loss: 0.006090488108963731


In [13]:
seq_classifier.save_model("./phish-bert-20211006")

## Evaluation of the Test Set of CLAIR+SPAM_ASSASSIN+ENRON Datasets

In [14]:
seq_classifier.evaluate_model(X_test["email"], y_test)

0.9968354430379747

## Conclusion

When all three datasets are combined, over 99% of the emails are labelled correctly. This shows that using a BERT-based phishing detector performs well in identifying the spam emails across these datasets. This notebook is prepared as a POC. Users can experiment with other datasets, increase the coverage and change the number of epochs to fine-tune the results on their datasets. 

# References
* Radev, D. (2008), CLAIR collection of fraud email, ACL Data and Code Repository, ADCR2008T001, http://aclweb.org/aclwiki
* https://www.kaggle.com/rtatman/fraudulent-email-corpus *
* https://www.cs.cmu.edu/~./enron/
* https://spamassassin.apache.org/old/publiccorpus/readme.html
* https://github.com/huggingface/transformers/tree/master/examples#
* https://www.depends-on-the-definition.com/named-entity-recognition-with-bert/
* https://github.com/ThilinaRajapakse/pytorch-transformers-classification
* https://mccormickml.com/2019/07/22/BERT-fine-tuning/